In [1]:
cd /home/c01muya/vae-disentanglement/disentanglement_lib_pl/

/home/c01muya/vae-disentanglement/disentanglement_lib_pl


In [2]:

import random
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import seaborn as sns

import torch
import torchvision.utils as vutils
import torchvision.transforms.functional as T
from torch.utils.data import DataLoader
from torchvision import transforms

from vae_experiment import VAEExperiment
from common.data_loader import DSpritesDataset, ThreeShapesDataset, ContinumDataset
from notebook_utils import *
from evaluation import evaluation_utils

# Change figure aesthetics
%matplotlib inline
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 1.5})


In [3]:
curr_dev = torch.device("cuda:2")
batch_size = 144
seed = 123

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [4]:
# load model
beta = 1.0
z_dim = 2
batch_size = 64
checkpoint_path="../pl-dt-test/3shapesnoisy_beta1/version_0/checkpoints/epoch=99-step=13999.ckpt"


acts_for_threeshapesn, threeshapesn_dataset, model_threeshapesn = load_model_and_data_and_get_activations(
                    dset_name='threeshapesnoisy', dset_path='../datasets', 
                    batch_size=batch_size, z_dim=z_dim , beta=beta, 
                    checkpoint_path=checkpoint_path, current_device=curr_dev, 
                    activation_with_label=True, seed=123,  batches=None
    )

100%|████████████████████████████████████████████| 140/140 [00:02<00:00, 52.41it/s]


In [6]:
import os
os.environ['DATASET_NAME'] = "threeshapesnoisy"
os.environ['DISENTANGLEMENT_LIB_DATA'] = "/home/c01muya/vae-disentanglement/datasets/"

evaluation_results = evaluation_utils.evaluate_disentanglement_metric(model_threeshapesn.model, 
                                                eval_results_dir=".",
                                                metric_names=['dci','mig'],
                                                dataset_name='threeshapesnoisy'
                            )

Getting non-builtin dataset [threeshapesnoisy] for eval... 


/home/c01muya/anaconda3/envs/pytorch-env/lib/python3.9/site-packages/scipy/stats/_entropy.py:82: RuntimeWarning: invalid value encountered in true_divide
  S /= np.log(base)


{'informativeness_train': 1.0, 'informativeness_test': 0.9994, 'disentanglement': nan, 'completeness': 0.7989257199470422, 'elapsed_time': 15.72533106803894, 'uuid': '3b8aa55f-60b1-4ae4-b0f6-331e0c36c35d'}
Getting non-builtin dataset [threeshapesnoisy] for eval... 
{'discrete_mig': 0.4201960447251299, 'elapsed_time': 6.02137017250061, 'uuid': '8c68ed83-ad1d-4a93-b4bc-d0751b848015'}


In [6]:
evaluation_results

{'eval_dci': 0.7989257199470422, 'eval_mig': 0.4201960447251299}

In [ ]:
from common.evaluation_datasets import ThreeShapesData
tsd = ThreeShapesData(is_noisy=True)
random_state = np.random.RandomState(seed)
tsd.sample(5, random_state)
fs = tsd.sample_factors(10, random_state)
N =10000
one_third = N // 3
shape_ranges = {
    'triangle': range(0, one_third),
    'square':   range(one_third, 2*one_third),
    'circle':   range(2*one_third, N)
}
t=np.array([
                [0, one_third],
                [one_third, 2*one_third],
                [2*one_third, N]
            ])

code_to_name = {0: 'triangle', 1: 'square', 2: 'circle'}
fs = tsd.sample_factors(10, random_state)
indices = [random_state.choice( range(t[shape_code.item()][0],t[shape_code.item()][1])) for shape_code in fs]